In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

In [ ]:
from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string
from utils.utils_chain import nominated_amount

context = Context()

In [ ]:
from contracts.farm_contract import FarmContract
from utils.contract_retrievers import retrieve_farm_by_address
# farm_contracts = context.get_contracts(config.FARMS_V2)

In [ ]:
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 0)

In [ ]:
farm_contract: FarmContract = retrieve_farm_by_address("")

set new rewards per block

In [ ]:
farm_contract.set_rewards_per_block(context.deployer_account, context.network_provider.proxy, nominated_amount(14000))

start contracts

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

produce rewards

In [ ]:
farm_contract.start_produce_rewards(context.deployer_account, context.network_provider.proxy)

upgrade contract

In [ ]:
pair_address = farm_contract.get_lp_address(context.network_provider.proxy)
tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                         config.FARM_V2_BYTECODE_PATH, [pair_address, context.deployer_account.address.bech32()])

change owner

In [ ]:
tx_hash = farm_contract.update_owner_or_admin(context.deployer_account, context.network_provider.proxy, "erd19uysnnzwnzm3klq8qzv7zprw2d3yj38hx0ac9cuqvl0j2jr5kp2sflvmyv")

resume contract

In [ ]:
farm_contract.resume(context.deployer_account, context.network_provider.proxy)

Collect user accounts

In [ ]:
from utils.utils_chain import WrapperAddress as Address, get_all_token_nonces_details_for_account
from multiprocessing.dummy import Pool
from multiversx_sdk_network_providers.api_network_provider import ApiNetworkProvider
from multiversx_sdk_network_providers.interface import IPagination
from multiversx_sdk_network_providers.transactions import TransactionOnNetwork
from typing import List

class FetchedUser:
    def __init__(self, address: Address, farming_tokens: list, farm_tokens: list) -> None:
        self.address = address
        self.farming_tokens = farming_tokens
        self.farm_tokens = farm_tokens

    def __str__(self) -> str:
        return f'Address: {self.address.bech32()}\nFarming tokens: {self.farming_tokens}\nFarm tokens: {self.farm_tokens}'

    def __repr__(self) -> str:
        return self.__str__()
    
class FetchedUsers:
    def __init__(self) -> None:
        self.users: List[FetchedUser] = []

    def add_user(self, user: FetchedUser) -> None:
        # check if user already exists
        if user.address in [user.address for user in self.users]:
            return
        self.users.append(user)

    # getter for users having farming tokens
    def get_users_with_farming_tokens(self) -> list:
        return [user for user in self.users if len(user.farming_tokens) > 0]
    
    # getter for users having farm tokens
    def get_users_with_farm_tokens(self) -> list:
        return [user for user in self.users if len(user.farm_tokens) > 0]
    
    # getter for users having more than one farm tokens
    def get_users_with_multiple_farm_tokens(self) -> list:
        return [user for user in self.users if len(user.farm_tokens) > 1]
    
    # getter for users having both farming and farm tokens
    def get_users_with_both_tokens(self) -> list:
        return [user for user in self.users if len(user.farming_tokens) > 0 and len(user.farm_tokens) > 0]

    def __str__(self) -> str:
        return '\n'.join([str(user) for user in self.users])

    def __repr__(self) -> str:
        return self.__str__()

class Pagination(IPagination):
    def __init__(self, start: int, size: int) -> None:
        self.start = start
        self.size = size

    def get_start(self) -> int:
        return self.start

    def get_size(self) -> int:
        return self.size

mainnet_api = ApiNetworkProvider("https://api.multiversx.com")
transactions = mainnet_api.get_account_transactions(Address(farm_contract.address), Pagination(0, 200))

fetched_users = FetchedUsers()

def process_tx(tx: TransactionOnNetwork):
    user = tx.sender
    print(f'Processing user {user.bech32()} ...')
    try:
        farming_tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.bech32(), context.network_provider.proxy)
        farm_tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.bech32(), context.network_provider.proxy)

        if len(farming_tokens_in_account) > 0 or len(farm_tokens_in_account) > 0:
            fetched_users.add_user(FetchedUser(user, farming_tokens_in_account, farm_tokens_in_account))
    except Exception as e:
        print(f'Error processing user {user.bech32()}: {e}')

Pool(10).map(process_tx, transactions)

print(f'Number of users fetched: {len(fetched_users.users)}')
print(f'Number of users with farming tokens: {len(fetched_users.get_users_with_farming_tokens())}')
print(f'Number of users with farm tokens: {len(fetched_users.get_users_with_farm_tokens())}')
print(f'Number of users with both farming and farm tokens: {len(fetched_users.get_users_with_both_tokens())}')

Used users

In [ ]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex
index = 0
users: List[FetchedUser] = fetched_users.get_users_with_farm_tokens()
if not users:
    raise Exception('No users found for the given criteria')

user = users[index].address
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.bech32(), context.network_provider.proxy)
print(f'Account: {user.bech32()}')
print(f'Farming Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.bech32(), context.network_provider.proxy)
print(f'Farm Tokens in account:')
for token in tokens_in_account:
    print(f'\t{token}')
    print(f'\t\t{decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)}')

Transfer position

In [ ]:
from utils.utils_tx import multi_esdt_transfer, ESDTToken
from utils.utils_chain import Account
fetched_user = users[0]
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = fetched_user.address
user_account.sync_nonce(context.network_provider.proxy)

full_token_id = fetched_user.farm_tokens[0].get('tokenIdentifier')
token_id = '-'.join(full_token_id.split('-')[:-1])
token = ESDTToken(token_id, fetched_user.farm_tokens[0].get('nonce'), int(fetched_user.farm_tokens[0].get('balance'))//2)
print(f'Sending: {token.get_token_data()} from {fetched_user.address.bech32()}')
multi_esdt_transfer(context.network_provider.proxy, 1000000, user_account, users[1].address, [token])

Enter farm

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farming_tk_balance = 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farming tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > 0:
        farming_tk_balance = int(token['balance'])
        break

if not farming_tk_balance:
    raise Exception("Not enough farming token balance")

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} farm tokens in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

# use only 10% of the balance
event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance // 10,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

initial_enter: bool = False if farm_tk_nonce else True
tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, initial=initial_enter)

Claim rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

Exit rewards

In [ ]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ExitFarmEvent

# user = fetched_users[randint(0, len(fetched_users.get_users_with_farming_tokens()) - 1)].address
# user = fetched_users.get_users_with_both_tokens()[0].address
user_account = Account(pem_file="~/Documents/sh1.pem")
user_account.address = user
user_account.sync_nonce(context.network_provider.proxy)

farm_tk_balance, farm_tk_nonce = 0, 0
tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
for token in tokens_in_account:
    if int(token['balance']) > farm_tk_balance:
        farm_tk_balance = int(token['balance'])
        farm_tk_nonce = token['nonce']
        break

if not farm_tk_nonce:
    raise Exception("Not enough farm token balance")

event = ExitFarmEvent(farm_contract.farmToken, farm_tk_balance, farm_tk_nonce, '')

tx_hash = farm_contract.exitFarm(context.network_provider, user_account, event)

Get user farm position

In [ ]:
# user_address = fetched_users.get_users_with_both_tokens()[0].address.bech32()
user_address = user_account.address.bech32()
print(user_address)
farm_contract.get_user_total_farm_position(user_address, context.network_provider.proxy)

SHADOWFORK CONTROL

In [ ]:
from contracts.builtin_contracts import SFControlContract

epocs_to_advance = 7

sf_control_contract = SFControlContract(config.SF_CONTROL_ADDRESS)
current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
print(f'Current epoch: {current_epoch}')
context.deployer_account.sync_nonce(context.network_provider.proxy)
sf_control_contract.epochs_fast_forward(context.deployer_account, context.network_provider.proxy, epocs_to_advance, 9)

wait for epoch fast forward

In [ ]:
from time import sleep

target_epoch = current_epoch + epocs_to_advance
while current_epoch < target_epoch:
    current_epoch = context.network_provider.proxy.get_network_status(1).epoch_number
    print(f'Current epoch: {current_epoch}')
    sleep(10)
print('Reached target epoch!')

Trackers

In [ ]:
from trackers.farm_economics_tracking import FarmEconomics
from contracts.farm_contract import FarmContractVersion

farm_tracker = FarmEconomics(farm_contract.address, FarmContractVersion.V2Boosted, context.network_provider)

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()

In [ ]:
farm_tracker.update_tracking_data()
farm_tracker.report_current_tracking_data()